# 필요한 라이브러리 import

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from gensim import corpora
from gensim.models import Word2Vec
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as spr
import configparser
import pymysql
from itertools import chain
from collections import defaultdict,Counter
import hnswlib
import warnings
import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from implicit.als import AlternatingLeastSquares
from sklearn.decomposition import NMF
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
import random
from scipy.stats import skew,kurtosis
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [ ]:
with open('./data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
song_df = pd.DataFrame.from_dict(song_dict)

In [ ]:
with open('./data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

In [ ]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

# 태그-id간 동기화를 위한 데이터베이스 연결

In [ ]:
# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 연결 config 변수저장

user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']

In [ ]:
# 데이터 베이스 연결
melon_recom_db = pymysql.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = int(port),
    cursorclass = pymysql.cursors.DictCursor
)

In [ ]:
# 커서 생성 후 쿼리문 수행
melon_cursor = melon_recom_db.cursor()
melon_cursor.execute('select tag_id,tag from tag')
tag_db_list=melon_cursor.fetchall()

# tag 딕셔너리 만들기 

In [ ]:
# tag를 id로
tag_to_id = {}

# id를 tag로
id_to_tag = {}

for tag_db in tag_db_list:
    tag_to_id[tag_db['tag']] = tag_db['tag_id']
    
for tag_db in tag_db_list:
    id_to_tag[tag_db['tag_id']] = tag_db['tag']

# tag 전처리 빈도수로 컷

In [ ]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [ ]:
# 태그 빈도의 분포를 알아보자
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")
tag_freq.columns=['빈도']
tag_freq.describe().T

In [ ]:
# filter된 태그의 id만 남기기
train_df['tag_ids'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])
train_df.head(1)

## 컬럼명 한글화

In [ ]:
train_df.columns=['태그','플리아이디','플리제목','노래들','좋아요수','변경일자','태그아이디']

In [ ]:
train_df['태그수'] = train_df['태그'].map(len)

In [ ]:
train_df.head(1)

# Train Test Split

In [ ]:
# 태그가 10개인거 부터 train에 쓸거임
origin_tags = train_df[train_df['태그수']>9]['태그'].tolist()

train_tags = []
test_tags = []

# 태그는 많아봐야 3개 사분위수 75% -> 3개
for tags in origin_tags:
    tag_3p = len(tags)//3
    train_tag = random.sample(tags, tag_3p)
    test_tag = list(set(tags)-set(train_tag))
    train_tags.append(train_tag)
    test_tags.append(test_tag)

In [ ]:
cal_tag_hit_df = pd.DataFrame(columns=['태그원본','예측용태그','검증용태그','als_예측결과','als_히트'])
cal_tag_hit_df['태그원본'] = origin_tags

In [ ]:
cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df.head()

# 희소행렬생성 태그,음악 행렬

In [ ]:
song_df.colums=['세부장르','발매일','앨범명','앨범ID','가수ID','노래명','대장르','가수명','노래ID']

In [ ]:
genre_dict['GN9000'] = '기타장르'

genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

genre_detail_dict = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        genre_detail_dict[k] = new_value

In [ ]:
genre_big_dict = {}

for k,v in genre_big.items():
    genre_big_dict[k+'00'] = v

# 태그 임베딩

## 대장르ID와 CODE간 딕셔너리 생성

In [ ]:
genre_big_id_to_code = {}

for i,v in enumerate(list(genre_big_dict.keys())):
    genre_big_id_to_code[i] = v

In [ ]:
genre_big_code_to_id = {}

for i,v in enumerate(list(genre_big_dict.keys())):
    genre_big_code_to_id[v] = i

In [ ]:
# 노래의 ID를 KEY 대\장르의 ID리스트를 ITEM으로 하는 딕셔너리 생성 

song_genre_big_dict = defaultdict(list)

for codes,id in zip(song_df['song_gn_gnr_basket'].tolist(),song_df['id'].tolist()):
    for code in codes:
        song_genre_big_dict[id].append(genre_big_code_to_id[code])

In [ ]:
# 플레이리스트의 모든 노래의 세부장르를 모아서 새로운 컬럼으로 생성

def fetcher(x):
    temp = []
    for song in x:
        genre_ids = song_genre_big_dict[song]
        for id in genre_ids:
            temp.append(id)
    return temp

train_df['대장르'] = train_df['노래들'].map(fetcher)
train_df.head(1)

## 태그ID와 대장르ID 간 딕셔너리 생성

In [ ]:
# {tag_Id:genre_detail_ids}

tag_id_to_genre_big_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['대장르'].tolist()):
    for id in ids:
        tag_id_to_genre_big_ids[id].extend(genres)

## 대장르의 비율로 데이터 전환

In [ ]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_big_ids.items():
    tag_id_to_genre_big_ids[k] = dict(Counter(v))

In [ ]:
# 딕셔너리의 빈도를 min_max_scaling하고 1을 더함 -> 0이 되는것은 싫다!
for k,value_dict in tag_id_to_genre_big_ids.items():
    try:
        max_val = np.max(list(value_dict.values()))
  
    min_val = np.min(list(value_dict.values()))
    for key,value in value_dict.items():
        value_dict[key] = np.round((value-min_val)/(max_val-min_val),3)

## 만든 딕셔너리를 이용해서 csr_matrix 생성

In [ ]:
# 행,열,데이터 list를 생성
row = []
col = []
dat = []

for k,v in tag_id_to_genre_big_ids.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

In [ ]:
len(genre_big)

In [ ]:
# 29160의 태그의 가짓수, 224는 세부장르의 가짓수
A = spr.csr_matrix((dat, (row, col)), shape=(29160, 31))

In [ ]:
# 자료형 변경
A = A.astype(float)

In [ ]:
# csr_matrix를 저장
spr.save_npz('./data/tag_genreb_csr.npz',A)

# Vectorization with SVD

## 태그-장르 행렬의 ALS 행렬분해 수행 

In [ ]:
als = AlternatingLeastSquares()
als.fit(A)
als.similar_items(tag_to_id['기분전환'])

## cosine유사도를 이용해서 KNN 모델 생성

In [ ]:
# 100차원으로 index 생성 및 초기화
svd_p = hnswlib.Index(space='cosine', dim=dim)  
svd_p.init_index(max_elements=data_len, ef_construction=100, M=100)

# 짐재행렬 추가
svd_p.add_items(svd_vectors,np.arange(data_len))

In [ ]:
#svd knn모델 저장
with open('./data/svd_knn_model.pickle', 'wb') as f:
    pickle.dump(svd_p,f)

In [ ]:
# svd tag의 벡터들을 저장
with open('./data/svd_tag_vectors.pickle', 'wb') as f:
    pickle.dump(svd_vectors,f)

In [ ]:
# svd_knn_model로 태그를 예측하여 예측된 결과의 list를 저장하는 함수

def tag_svd(x):
    target_ids = [tag_to_id[t] for t in x]
    
    vectors = np.zeros((100,1))
    
    for id in target_ids:
        vectors = vectors+svd_vectors[id]
    
    labels, distances = svd_p.knn_query(vectors/len(target_ids), k = 10+len(x))

    ids = [label for label in labels[0]]
    
    return list(set([id_to_tag[tag] for tag in ids])-set(x))

In [ ]:
# 예측수행
cal_tag_hit_df['svd_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_svd)
cal_tag_hit_df.head()

In [ ]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
svd_tag = cal_tag_hit_df['svd_예측결과'].tolist()

In [ ]:
svd_hit = []

for val,svd in zip(val_tag,svd_tag):
    if len(val) == len(set(val)-set(svd)):
        svd_hit.append(0)
    else:
        svd_hit.append(1)

In [ ]:
cal_tag_hit_df['svd_히트'] = svd_hit

In [ ]:
cal_tag_hit_df

In [ ]:
sum(svd_hit)/len(svd_hit)

In [ ]:
cal_tag_hit_df.head(30)

## 과연 한 태그의 장르 점유율은 어떨까?

In [ ]:
# {tag_Id:genre_detail_ids}

tag_id_to_genre_detail_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['세부장르'].tolist()):
    for id in ids:
        tag_id_to_genre_detail_ids[id].extend(genres)

In [ ]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_detail_ids.items():
    tag_id_to_genre_detail_ids[k] = dict(Counter(v))

In [ ]:
tag_genre_sorted = {}
for k,v in tag_id_to_genre_detail_ids.items():
    tag_genre_sorted[k] = sorted(tag_id_to_genre_detail_ids[k].items(),key=lambda x : x[1],reverse=True)

In [ ]:
genre_counts = []

for k,v in tag_genre_sorted.items():
    genre_counts.append([count for id,count in v])

In [ ]:
genre_skew = list(map(lambda x : skew(x),genre_counts))

In [ ]:
genre_kurtosis = list(map(lambda x : kurtosis(x),genre_counts))

In [ ]:
skew(genre_counts[2])

In [ ]:
np.median(genre_counts[2])

In [ ]:
max(genre_kurtosis), min(genre_kurtosis)

In [ ]:
np.median(genre_kurtosis)

In [ ]:
genre_kurtosis.index(min(genre_kurtosis))

In [ ]:
genre_kurtosis

In [ ]:
len(genre_skew)

In [ ]:
#sns.kdeplot([(x-np.mean(genre_counts[0]))/np.std(genre_counts[0]) for x in genre_counts[0]],shade=True)
#sns.kdeplot(np.random.normal(),shade=True)
#np.mean(genre_counts[2])

#ax = sns.kdeplot(genre_counts[2],shade=False,color='crimson')
ax = sns.distplot(genre_counts[1099],color='crimson')
kdeline = ax.lines[0]
mean = np.mean(genre_counts[1099])
xs = kdeline.get_xdata()
ys = kdeline.get_ydata()
height = np.interp(mean,xs,ys)
ax.vlines(mean,0,height,color='crimson',ls=':')
ax.fill_between(xs,0,ys,facecolor='crimson',alpha=0.2)
plt.show()
print(f' 평균 : {mean}  왜도: {skew(genre_counts[1099])} 첨도: {kurtosis(genre_counts[1099])}')

In [ ]:
id_to_tag[1099]

In [ ]:
tag_id_to_genre_big_ids[2]

In [ ]:
genre_counts_pie = []

genre_counts_pie.extend(genre_counts[2])
genre_counts_pie.append(np.sum(genre_counts[2][10:]))

plt.pie(genre_counts_pie)

plt.show()

# 과연 한 태그의 대장르의 분포가 치우쳐져 있을까?

In [ ]:
# {tag_Id:genre_big_ids}

tag_id_to_genre_big_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['대장르'].tolist()):
    for id in ids:
        tag_id_to_genre_big_ids[id].extend(genres)

In [ ]:
# 대장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_big_ids.items():
    tag_id_to_genre_big_ids[k] = dict(Counter(v))

In [ ]:
tag_big_genre_sorted = {}
for k,v in tag_id_to_genre_big_ids.items():
    tag_big_genre_sorted[k] = sorted(tag_id_to_genre_big_ids[k].items(),key=lambda x : x[1],reverse=True)

In [ ]:
big_genre_counts = []

for k,v in tag_big_genre_sorted.items():
    big_genre_counts.append([count for id,count in v])

In [ ]:
big_genre_counts_pie = []

big_genre_counts_pie.extend(big_genre_counts[2])
#big_genre_counts_pie.append(np.sum(big_genre_counts[2][5:]))

plt.pie(big_genre_counts_pie)

plt.show()

# 과연 플레이리스트안의 세부장르의 분포는?

In [ ]:
train_df['새플리아이디'] = train_df.index
train_df.head(1)

In [ ]:
p_id_to_genre_detail_ids = defaultdict(list)

for id,genres in zip(train_df['새플리아이디'].tolist(),train_df['세부장르'].tolist()):
    p_id_to_genre_detail_ids[id].extend(genres)

In [ ]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in p_id_to_genre_detail_ids.items():
    p_id_to_genre_detail_ids[k] = dict(Counter(v))

In [ ]:
p_genre_sorted = {}
for k,v in p_id_to_genre_detail_ids.items():
    p_genre_sorted[k] = sorted(p_id_to_genre_detail_ids[k].items(),key=lambda x : x[1],reverse=True)

In [ ]:
p_genre_counts = []

for k,v in p_genre_sorted.items():
    p_genre_counts.append([count for id,count in v])

In [ ]:
p_genre_skew = list(map(lambda x : skew(x),p_genre_counts))

In [ ]:
max(p_genre_skew), min(p_genre_skew)

In [ ]:
p_genre_kurtosis = list(map(lambda x : kurtosis(x),p_genre_counts))

In [ ]:
max(p_genre_kurtosis), min(p_genre_kurtosis)